In [ ]:
# 🧪 Python Kodēšana 2 – Tests (1.–4. nodarbība)
import ipywidgets as widgets
from IPython.display import display, clear_output, Javascript
import random
from datetime import datetime
import requests

# Global state
submitted = False

# Jautājumi
questions_data = [
    ("Ko dara pygame.init()?", [
        "Inicializē visus importētos pygame moduļus",
        "Palaiž spēles ciklu",
        "Ielādē spēles logu",
        "Izveido galveno varoņa sprite objektu"
    ], 0),
    ("Kura metode tiek izmantota, lai uzzīmētu taisnstūri pygame logā?", [
        "pygame.draw.rect()",
        "pygame.screen.rectangle()",
        "pygame.create.rect()",
        "pygame.render.rect()"
    ], 0),
    ("Kā tiek atjaunināts logs pygame pēc zīmēšanas?", [
        "pygame.display.flip()",
        "pygame.update.display()",
        "pygame.screen.refresh()",
        "pygame.render.frame()"
    ], 0),
    ("Ko pārbauda event.type == pygame.QUIT?", [
        "Vai lietotājs ir nospiedis loga aizvēršanas pogu",
        "Vai lietotājs nospieda taustiņu Enter",
        "Vai spēlei jāsākas no jauna",
        "Vai spēlētājs zaudēja"
    ], 0),
    ("Kurš modulis tiek izmantots, lai veidotu GUI lietotnes Python valodā?", [
        "tkinter",
        "pygame.gui",
        "guimodule",
        "tkbuilder"
    ], 0),
    ("Kurš tkinter logelements tiek izmantots, lai saņemtu tekstu no lietotāja?", [
        "Entry",
        "Label",
        "Button",
        "Canvas"
    ], 0),
    ("Ko dara mainloop() tkinter lietotnē?", [
        "Palaiž GUI notikumu ciklu",
        "Pieslēdzas datubāzei",
        "Atjauno Python vidi",
        "Izveido loga izvietojumu"
    ], 0),
    ("Kurš tkinter logelements ļauj zīmēt figūras kā taisnstūrus un tekstu?", [
        "Canvas",
        "Frame",
        "Box",
        "Entry"
    ], 0)
]

random.shuffle(questions_data)
results = [None] * len(questions_data)
attempts = [0] * len(questions_data)
question_widgets = []
submit_button = widgets.Button(description="Iesniegt rezultātu", button_style='success')
download_button = widgets.Button(description="⬇️ Lejupielādēt rezultātu", button_style='info')
name_input = widgets.Text(description='Vārds:', placeholder='Ievadi savu vārdu')

# Google Forms konfigurācija
form_url = "https://docs.google.com/forms/d/e/1FAIpQLSdTSrnEP9GQajJRRCKqQMbwWGS_TL3o3B4T_fequkWmUZQCCg/formResponse"
entry_ids = {
    "Students": "entry.1649720525",
    "Datums": "entry.667959859",
    "Rezultats": "entry.940605140",
    "1": "entry.213159259",
    "2": "entry.267050949",
    "3": "entry.418893037",
    "4": "entry.1447996441",
    "5": "entry.1914428828",
    "6": "entry.1728217231",
    "7": "entry.1875638404",
    "8": "entry.907839017"
}

# Timer setup
timer_output = widgets.Output()
refresh_button = widgets.Button(description="Atjaunot taimeri", button_style='warning')
minutes = 30
seconds = minutes * 60
start_time = datetime.now().timestamp()

def update_timer():
    global submitted, timer_text
    if submitted:
        return
    elapsed = datetime.now().timestamp() - start_time
    remaining = max(0, seconds - int(elapsed))
    if remaining <= 0 and not submitted:
        timer_text = "Laiks beidzies! Rezultāti nosūtīti."
        with timer_output:
            clear_output()
            display(widgets.HTML(f"<b>{timer_text}</b>"))
        send_to_google_form(None)
        return
    mins, secs = divmod(remaining, 30)
    timer_text = f"{mins:02d}:{secs:02d}"
    with timer_output:
        clear_output()
        display(widgets.HTML(f"<b>Atlikušais laiks: {timer_text}</b>"))
    display(Javascript('setTimeout(() => { IPython.notebook.kernel.execute("update_timer()"); }, 1000);'))

def refresh_timer(_):
    update_timer()

refresh_button.on_click(refresh_timer)

def create_question(index, question, options, correct_index):
    out = widgets.Output()
    paired = list(enumerate(options))
    random.shuffle(paired)
    shuffled_indexes, shuffled_options = zip(*paired)
    correct_shuffled_index = shuffled_indexes.index(correct_index)

    radio = widgets.RadioButtons(options=shuffled_options)
    button = widgets.Button(description='Pārbaudīt')
    feedback = widgets.HTML()

    def on_click(b):
        global submitted
        if submitted:
            return
        attempts[index] += 1
        if radio.index == correct_shuffled_index:
            penalty = 5 * (attempts[index] - 1)
            score = max(12.5 - penalty, 0)
            results[index] = score
            feedback.value = "<span style='color:green'>✔️ Pareizi!</span>"
            radio.disabled = True
            button.disabled = True
            if all(r is not None for r in results):
                display(widgets.VBox([name_input, submit_button]))
        else:
            feedback.value = "<span style='color:red'>❌ Nepareizi. Mēģini vēlreiz.</span>"

    button.on_click(on_click)
    with out:
        display(widgets.HTML(f"<b>{index+1}. {question}</b>"), radio, button, feedback)
    return out

def calculate_score(_):
    global submitted
    if submitted:
        return
    submitted = True
    clear_output()
    for w in question_widgets:
        display(w)
    total = sum(r if r else 0 for r in results)
    final_score = round(total)
    status = "✅ Nokārtots!" if final_score >= 70 else "❌ Nenokārtots."
    color = "green" if final_score >= 70 else "red"
    display(widgets.HTML(f"<h3>Rezultāts: <span style='color:{color}'>{final_score}%</span> – {status}</h3>"))
    display(widgets.VBox([download_button, send_button]))

def download_result(_):
    student_name = name_input.value.strip() or "Nezināms"
    total = sum(r if r else 0 for r in results)
    final_score = round(total)
    now = datetime.now().strftime("%Y-%m-%d %H:%M")
    filename = f"rezultats_{student_name.replace(' ', '_')}.txt"
    with open(filename, 'w', encoding='utf-8') as f:
        f.write(f"Students: {student_name}\nDatums: {now}\nRezultāts: {final_score}%\n\n")
        for i, (q, _, _) in enumerate(questions_data):
            f.write(f"{i+1}. {q} (Mēģinājumi: {attempts[i]}) – {results[i] if results[i] is not None else 0} punkti\n")
    try:
        from google.colab import files
        files.download(filename)
    except:
        print(f"Fails saglabāts kā {filename}")

def send_to_google_form(_):
    global submitted
    if submitted:
        return
    submitted = True
    student_name = name_input.value.strip() or "Nezināms"
    total = sum(r if r else 0 for r in results)
    final_score = round(total)
    now = datetime.now().strftime("%Y-%m-%d %H:%M")
    form_data = {
        entry_ids["Students"]: student_name,
        entry_ids["Datums"]: now,
        entry_ids["Rezultats"]: str(final_score) + "%"
    }
    for i, (q, _, _) in enumerate(questions_data):
        question_entry = entry_ids[str(i+1)]
        response = f"{q} (Mēģinājumi: {attempts[i]}) – {results[i] if results[i] is not None else 0} punkti"
        form_data[question_entry] = response
    try:
        response = requests.post(form_url, data=form_data, timeout=10)
        with timer_output:
            clear_output()
            if response.status_code == 200:
                display(widgets.HTML("<h4 style='color:green;'>Rezultāti veiksmīgi nosūtīti uz Google Formu!</h4>"))
            else:
                display(widgets.HTML(f"<h4 style='color:red;'>Kļūda sūtot datus (statuss: {response.status_code})</h4>"))
    except Exception as e:
        with timer_output:
            clear_output()
            display(widgets.HTML(f"<h4 style='color:red;'>Kļūda sūtot datus: {e}</h4>"))

submit_button.on_click(calculate_score)
download_button.on_click(download_result)
send_button = widgets.Button(description="📤 Sūtīt uz Google Formu", button_style='primary')
send_button.on_click(send_to_google_form)

# Parāda taimeri un jautājumus
display(refresh_button)
display(timer_output)
update_timer()
for i, (q, opts, correct) in enumerate(questions_data):
    qw = create_question(i, q, opts, correct)
    question_widgets.append(qw)
    display(qw)
